In [ ]:
!pip install nlpaug
!pip install transformers
!pip install torch torchvision

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

from nlpaug.util.file.download import DownloadUtil
# DownloadUtil.download_word2vec(dest_dir='.') # Download word2vec model
# DownloadUtil.download_glove(model_name='glove.6B', dest_dir='.') # Download GloVe model
# DownloadUtil.download_fasttext(model_name='wiki-news-300d-1M', dest_dir='.') # Download fasttext model
!pip install simpletransformers>=0.61.10
!pip install torch>=1.6.0 transformers>=4.11.3 sentencepiece
!pip install sacremoses
!pip install gensim>=4.1.2
!pip install nltk>=3.4.5

In [ ]:
#source for augmenter https://nlpaug.readthedocs.io/en/latest/augmenter/word/synonym.html
aug = naw.SynonymAug(aug_src='wordnet')

augmented_text = aug.augment(text)

df['synonym_aug'] = df['text'].apply(lambda x: aug.augment(x))

#backtranslation 
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en',  device='cuda'
)

def backtranslate(x):
  trans = back_translation_aug.augment(x)
  print(trans)
  return trans

df['translate_aug'] = df['text'].apply(lambda x, i = 0: backtranslate(x))

aug = naw.ContextualWordEmbsAug(
    model_path='roberta-large', action="substitute", device='cuda')
aug2 = naw.ContextualWordEmbsAug(
    model_path='roberta-large', action="insert", device='cuda')
augmented_text = aug.augment(text)

def contextual_aug(text):
  return aug.augment(aug2.augment(text))

df['contextual_aug'] = df['text'].apply(lambda x: contextual_aug(x))

In [ ]:
import nlpaug.augmenter.word as naw

# Define the SynonymAug augmenter with the WordNet thesaurus as the source
syn_aug = naw.SynonymAug(aug_src='wordnet')

# Apply the SynonymAug augmenter to each text in the dataframe and save the augmented text in a new column
df['synonym_aug'] = df['text'].apply(lambda x: syn_aug.augment(x))

# Define the BackTranslationAug augmenter with the pre-trained translation models
bt_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en', 
    device='cuda'
)

# Define a function to apply the BackTranslationAug augmenter to each text in the dataframe
def backtranslate(x):
    trans = bt_aug.augment(x)
    # print(trans)
    return trans

# Apply the backtranslate function to each text in the dataframe and save the augmented text in a new column
df['translate_aug'] = df['text'].apply(backtranslate)

# Define the ContextualWordEmbsAug augmenter with the pre-trained RoBERTa model and the "substitute" action
cont_aug_sub = naw.ContextualWordEmbsAug(
    model_path='roberta-large', 
    action="substitute", 
    device='cuda'
)

# Define the ContextualWordEmbsAug augmenter with the pre-trained RoBERTa model and the "insert" action
cont_aug_ins = naw.ContextualWordEmbsAug(
    model_path='roberta-large', 
    action="insert", 
    device='cuda'
)

# Define a function to apply both ContextualWordEmbsAug augmenters to each text in the dataframe
def contextual_aug(text):
    return cont_aug_ins.augment(cont_aug_sub.augment(text))

# Apply the contextual_aug function to each text in the dataframe and save the augmented text in a new column
df['contextual_aug'] = df['text'].apply(contextual_aug)

df['syn_and_contextual_aug'] = df['text'].apply(lambda x: contextual_aug(syn_augment(x)))